# PROYECTO INDIVIDUAL
# Data Engineering

## Procesando los diferentes datasets.

Se comienza a procesar los distitos archivos extraidos de la siguiente liga:
https://drive.google.com/drive/folders/1Rsq-HHomPtQwy7RIWQ574wKcf56LiGq1?usp=sharing


1.Se verifica que los distintos archivos que conforman el dataset tienen distintos formatos, por lo que se procede a realizar la tranformación de los archivos a formato CSV y estandarizarlos a UTF-8, para poder cargalos a un base de datos de MySQL.

### Tabla producto
Comenzamos con la tabla de productos que viene en formato ".parquet", se realiza un análisis exploratorio de esta tabla.

In [198]:
#Se exportan las diferentes librerias a utilizar durante el procesamiento de los datos
import pandas as pd
import numpy as np
import datetime as dt
from encodings import utf_8

In [199]:
#Se extrae el archivo de producto.parquet
producto = pd.read_parquet(".\Datasets\producto.parquet")

#Convertimos a dataframe y comenzamos a normalizar los datos, se verifica estructura de la tabla
producto= pd.DataFrame(producto)
producto
#producto.info()  ##verificamos el formato de los datos

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un,None,None,None
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un,None,None,None
2,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg,None,None,None
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito 1 Kg,1.0 kg,None,None,None
4,0000000205955,SIN MARCA,Chiquizuela Novillito 1 Kg,1.0 kg,None,None,None
...,...,...,...,...,...,...,...
72033,9569753142128,DELI-SITAS,Milhojas Cobertura de Chocolate Blanco Deli-Si...,500.0 gr,None,None,None
72034,9795403001143,MAYO,Mini Pizzetas Mayo 12 Un,12.0 un,None,None,None
72035,9990385651922,DANA,Te Negro en Hebras Lata Dana 50 Gr,50.0 gr,None,None,None
72036,9990385651939,DANA,Te Verde en Hebras Lata Dana 50 Gr,50.0 gr,None,None,None


Se determina que la columna "id"  contiene algunas incosistencias debido a que algunos id de los productos tienen parte del código de "sucursal_id" que se relaciona con otras tablas, con el fin de normalizar los datos y poder realizar las consultas, se cambiara el nombre de la columna "id" por "IdProducto" y se quitara eda parte del código de "sucursal_id", ademas de estandarizar el código a 13 dígitos. . Tambien, en las columnas "categoria1", "categoria2" y "categoria3" se colocara la etiqueta "sin categoria" en los valores nulos.

In [200]:
#Separamos los datos de Id y creo la columna de IdProducto
producto[['0','1', 'IdProducto']] = producto.id.str.rpartition('-',expand=True)

#Coloco etiqueta "Sin categoria" en las columnas "categoria1", "categoria2" y "categoria3"
# producto = producto.assign (categoria1 ="Sin categoria")
# producto = producto.assign (categoria2 ="Sin categoria")
# producto = producto.assign (categoria3 ="Sin categoria")
# precios.precio.fillna(0, inplace= True)
producto.categoria1.fillna('sin categoria', inplace=True)
producto.categoria2.fillna('sin categoria', inplace=True)
producto.categoria3.fillna('sin categoria', inplace=True)

#Normalizamos la columna IdProdcuto a 13 dígitos
# converting to string dtype
# producto["IdProducto"]= producto["IdProducto"].astype(str)
producto["IdProducto"] = producto["IdProducto"].str.zfill(13)

#Verificamos datos, no hay valores nulos o duplicados que se hayan detectado
#producto.dropna()               #solo son 2 resgistros
#producto.IdProducto.duplicated  #no detectados

#Seleccionamos las columnas que se guardaran en nuestro archivo.csv
producto = producto[['IdProducto', 'marca', 'nombre', 'presentacion', 'categoria1', 'categoria2', 'categoria3']]




producto

,IdProducto,marca,nombre,presentacion,categoria1,categoria2,categoria3
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un,sin categoria,sin categoria,sin categoria
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un,sin categoria,sin categoria,sin categoria
2,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg,sin categoria,sin categoria,sin categoria
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito 1 Kg,1.0 kg,sin categoria,sin categoria,sin categoria
4,0000000205955,SIN MARCA,Chiquizuela Novillito 1 Kg,1.0 kg,sin categoria,sin categoria,sin categoria
...,...,...,...,...,...,...,...
72033,9569753142128,DELI-SITAS,Milhojas Cobertura de Chocolate Blanco Deli-Si...,500.0 gr,sin categoria,sin categoria,sin categoria
72034,9795403001143,MAYO,Mini Pizzetas Mayo 12 Un,12.0 un,sin categoria,sin categoria,sin categoria
72035,9990385651922,DANA,Te Negro en Hebras Lata Dana 50 Gr,50.0 gr,sin categoria,sin categoria,sin categoria
72036,9990385651939,DANA,Te Verde en Hebras Lata Dana 50 Gr,50.0 gr,sin categoria,sin categoria,sin categoria


In [201]:
# producto.sort_values('IdProducto', ascending=False).drop_duplicates('A').sort_index()
producto.drop_duplicates('IdProducto', inplace=True)

C:\Users\Polaris Coneja\AppData\Local\Temp\ipykernel_13972\3941092135.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  producto.drop_duplicates('IdProducto', inplace=True)


In [202]:
#Guardamor en archivo en el formato .csv
#producto.to_csv(".\Datasets_procesados\producto_m.csv", index=False, sep=',', decimal='.', encoding= "utf_8")
producto.to_csv(r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\PIndividual\producto_m.csv", index=False, sep=',', decimal='.', encoding= "utf_8")

### Tabla sucursal

En la tabla sucursal esta en formato  .csv es UTF-8. No se detectan cambios signicativos que se tengan que realizar a este archivo. Solo se detectan 2 valores nulos y se renombra una de las columnas

In [217]:
# Para el archivo sucursal.csv
sucursal = pd.read_csv(".\Datasets\sucursal.csv")
sucursal


,id,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo
0,1-1-7,1,1,Super MAMI,Dinosaurio S.A.,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado
1,10-1-1,10,1,Hipermercado Carrefour,INC S.A.,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado
2,10-1-10,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado
3,10-1-11,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado
4,10-1-112,10,1,Hipermercado Carrefour,INC S.A.,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado
...,...,...,...,...,...,...,...,...,...,...,...,...
2328,9-3-5277,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-U,COMODORO RIVADAVIA,Yrigoyen Hipolito 0,-45.873300,-67.493500,Jumbo-Comodoro,Hipermercado
2329,9-3-5626,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-B,GENERAL PACHECO,Boulogne Sur Mer 1275,-34.474500,-58.625700,Jumbo Pacheco Novo,Hipermercado
2330,9-3-5961,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-C,CIUDAD AUTONOMA BUENOS AIRES,Avenida Santa Fe 4950,-34.577200,-58.430000,Jumbo Av. Santa Fé,Supermercado
2331,9-3-628,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-B,SAN FERNANDO,Avenida Del Libertador Gral San Martin 2271,-34.446900,-58.545700,Jumbo San Fernando,Supermercado


In [218]:
#Convertimos a dataframe
sucursal= pd.DataFrame(sucursal)

#Verificamos datos, no hay valores nulos o duplicados que se hayan detectado
# sucursal.dropna()
# sucursal.duplicated

#Cambiamos el nombre de la colunma id por IdSucursal
sucursal = sucursal.rename(columns= {'id': 'IdSucursal'})

# Guardamos el archivo en el formato correspondiente
# sucursal.to_csv('.\Datasets_procesados\sucursal_m.csv', index=False, sep=',', decimal='.', encoding= "utf_8")
sucursal.to_csv(r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\PIndividual\sucursal_m.csv", index=False, sep=',', decimal='.', encoding= "utf_8")

### TABLA PRECIOS

Se unificaran las tablas precios_semana_20200413.csv,precios_semana_20200503.json y precios_semana_20200419_20200426.xlsx para poder reunirlas en una sola tabla general llamada "precios"

Comienzo con la tabla precios_semana_20200413.csv tiene un encoding es UTF-16 y se tiene que modificar a UTF-8.
Se cambian los nombres de las columnas relacionadas con el IdSucursal y IdProducto. Se agregara la columna fecha y se normaliza la columna IdProducto a 13 dígitos.

In [206]:
# Para el archivo de precios_semana_20200413.csv

# Lectura del archivo csv
precios_20200413 = pd.read_csv("./Datasets/precios_semana_20200413.csv", encoding=  " utf-16")

#Convertimos a dataframe
precios_20200413 = pd.DataFrame(precios_20200413)


# #  Verificamos los datos
precios_20200413.duplicated    #  no se detectan valores duplicados
precios_20200413.dropna    # no hay valores nulos

# converting to string dtype
precios_20200413["producto_id"]= precios_20200413["producto_id"].astype(str)

# Separamos los datos de Id y creo la columna de IdProducto
precios_20200413[['0','1', 'IdProducto']] = precios_20200413.producto_id.str.rpartition('-',expand=True)
precios_20200413

# Se seleccionan las columnas
precios_20200413 = precios_20200413[['precio', 'IdProducto', 'sucursal_id']]
precios_20200413

# #Normalizamos la columna IdProducto a 13 dígitos
precios_20200413["IdProducto"] = precios_20200413["IdProducto"].str.zfill(13)


# Agregamos la columna de fecha
precios_20200413 = precios_20200413.assign (Fecha ="2020-04-13")

# Cambiamos el nombre la columna sucursal_id
precios_20200413 = precios_20200413.rename(columns={'sucursal_id': 'IdSucursal'})

# #Guardamos el archivos y cambiamos la codificación
# precios_20200413.to_csv ('.\Datasets_procesados\precios_semana_20200413_m.csv', index=False, sep=',', decimal='.', encoding= "utf_8")
producto.to_csv(r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\PIndividual\precios_semana_20200413_m.csv", index=False, sep=',', decimal='.', encoding= "utf_8")
precios_20200413

,precio,IdProducto,IdSucursal,Fecha
0,29.90,0000000001663,2-1-014,2020-04-13
1,29.90,0000000002288,2-1-032,2020-04-13
2,39.90,0000000002288,2-1-096,2020-04-13
3,499.99,0000000205870,9-1-686,2020-04-13
4,519.99,0000000205870,9-2-248,2020-04-13
...,...,...,...,...
472161,139.99,9569753142128,25-1-1,2020-04-13
472162,34.99,9795403001143,25-1-1,2020-04-13
472163,312.50,9990385651922,5-1-3,2020-04-13
472164,312.50,9990385651939,5-1-3,2020-04-13


Para la tabla precios_semana_20200503.json, tambien se verificaran los datos, se cambia nombre de las columnas y se normalizan los datos de la columna producto_id.

In [207]:
# Para el archivo de precios_semana_20200503.json
# import json
precios_20200503 = pd.read_json(".\Datasets\precios_semana_20200503.json")

#Convertimos a dataframe
# precios_20200503 = pd.DataFrame(precios_20200503)
# precios_20200503

# #  Verificamos los datos
# precios_20200503.duplicated    #  no se detectan valores duplicados
precios_20200503.dropna         # solo un valor nulo
# precios_20200503

# converting to string dtype
precios_20200503["producto_id"]= precios_20200503["producto_id"].astype(str)

# Separamos los datos de Id y creo la columna de IdProducto
precios_20200503[['0','1','IdProducto']] = precios_20200503.producto_id.str.rpartition('-', expand=True)
# precios_20200503

# #Normalizamos la columna IdProducto a 13 dígitos
precios_20200503["IdProducto"] = precios_20200503["IdProducto"].str.zfill(13)

# Se seleccionan las columnas
precios_20200503 = precios_20200503[['precio', 'IdProducto', 'sucursal_id']]
# precios_20200503

# Agregamos la columna de fecha
precios_20200503 = precios_20200503.assign (Fecha ="2020-05-03")

# Cambiamos el nombre la columna sucursal_id
precios_20200503 = precios_20200503.rename(columns={'sucursal_id': 'IdSucursal'})
# precios_20200503


#Guardamos el archivos y cambiamos la codificación
# precios_20200503.to_csv ('.\Datasets_procesados\precios_semana_20200503_m.csv', index=False, sep=',', decimal='.', encoding= "utf_8")
producto.to_csv(r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\PIndividual\precios_semana_20200503_m.csv", index=False, sep=',', decimal='.', encoding= "utf_8")
precios_20200503

,precio,IdProducto,IdSucursal,Fecha
0,29.9,0000000002288,2-1-187,2020-05-03
1,39.9,0000000002288,2-3-247,2020-05-03
2,499.99,0000000205870,9-1-685,2020-05-03
3,539.99,0000000205870,9-2-22,2020-05-03
4,519.99,0000000205870,9-2-59,2020-05-03
...,...,...,...,...
397729,139.99,9569753142128,25-1-1,2020-05-03
397730,34.99,9795403001143,25-1-1,2020-05-03
397731,312.5,9990385651922,5-1-5,2020-05-03
397732,312.5,9990385651939,5-1-5,2020-05-03


Para e archivo de excel se debe dividir el archivo en dos partes, ya que se encuentras en diferentes hojas y contienen diferentes tipo de errores.

In [208]:
# Para el archivo de precios_semanas_20200419_20200426.xlsx
#Cargando solo la hoja 2
precios_20200419 = pd.read_excel(".\Datasets\precios_semanas_20200419_20200426.xlsx", sheet_name = "precios_20200419_20200419")

In [209]:
# converting to string dtype y normalizando la cantidad numeros a 13 en la columna producto_id
precios_20200419["producto_id"]= precios_20200419["producto_id"].astype(str)
precios_20200419["producto_id"] = precios_20200419.producto_id.str.zfill(13)
precios_20200419

,precio,sucursal_id,producto_id
0,29.90,2-1-184,0000000002288
1,39.90,2-1-206,0000000002288
2,499.99,9-1-430,0000000205870
3,539.99,9-2-107,0000000205870
4,539.99,5218-03-09 00:00:00,0000000205870
...,...,...,...
458538,139.99,2001-01-25 00:00:00,9569753142128
458539,34.99,2001-01-25 00:00:00,9795403001143
458540,312.50,2003-01-05 00:00:00,9990385651922
458541,312.50,2003-01-05 00:00:00,9990385651939


En la columna sucursal_id se encuentran algunos datos con formato datatime, por lo cual se tiene que normalizar los datos. Se procede a realizar un funcion que permita obtener el formato deseado.

In [210]:
## Para cambiar los datos de sucursal_Id, se requiere quitar datos que estan mal capturados y normalizar la Id sucursal
try:
    cont = -1
    precios_20200419["nuevo"] = ""
    for a in precios_20200419["sucursal_id"]:
        cont += 1
        if type (a) == dt.datetime:
            precios_20200419["nuevo"].iloc[cont] = precios_20200419["sucursal_id"].iloc[cont].strftime("%d-%m-%Y")
    df = precios_20200419 ["nuevo"].str.split("-", expand = True)
    df = df.replace ("", None)
    df = df.fillna(value = 0).astype(int)
    precios_20200419["sucursal_id"][df[2] != 0] = df[[0,1,2]].astype(str).apply (lambda x: "-".join(x), axis = 1)
except:
    pass
del precios_20200419["nuevo"]

C:\Users\Polaris Coneja\AppData\Local\Temp\ipykernel_13972\435420983.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precios_20200419["nuevo"].iloc[cont] = precios_20200419["sucursal_id"].iloc[cont].strftime("%d-%m-%Y")
C:\Users\Polaris Coneja\AppData\Local\Temp\ipykernel_13972\435420983.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precios_20200419["sucursal_id"][df[2] != 0] = df[[0,1,2]].astype(str).apply (lambda x: "-".join(x), axis = 1)


In [211]:
# Cambio de las etiquetas en las columnas
precios_20200419 = precios_20200419.rename(columns={'sucursal_id': 'IdSucursal','producto_id' : 'IdProducto' })

# Agregando la fecha 
precios_20200419 = precios_20200419.assign (Fecha ="2020-04-19")

#Guardamos el archivos y cambiamos la codificación
# precios_20200419.to_csv ('.\Datasets_procesados\precios_semana_20200419_m.csv', index=False, sep=',', decimal='.', encoding= "utf_8")
producto.to_csv(r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\PIndividual\precios_semana_20200419_m.csv", index=False, sep=',', decimal='.', encoding= "utf_8")
precios_20200419

,precio,IdSucursal,IdProducto,Fecha
0,29.90,2-1-184,0000000002288,2020-04-19
1,39.90,2-1-206,0000000002288,2020-04-19
2,499.99,9-1-430,0000000205870,2020-04-19
3,539.99,9-2-107,0000000205870,2020-04-19
4,539.99,9-3-5218,0000000205870,2020-04-19
...,...,...,...,...
458538,139.99,25-1-2001,9569753142128,2020-04-19
458539,34.99,25-1-2001,9795403001143,2020-04-19
458540,312.50,5-1-2003,9990385651922,2020-04-19
458541,312.50,5-1-2003,9990385651939,2020-04-19


In [212]:
# Para el archivo de precios_semanas_20200419_20200426.xlsx
#Cargando solo la hoja 2
precios_20200426 = pd.read_excel(".\Datasets\precios_semanas_20200419_20200426.xlsx", sheet_name = "precios_20200426_20200426" )
precios_20200426


,precio,sucursal_id,producto_id
0,399.0,2-1-092,2.288000e+03
1,299.0,2-1-206,2.288000e+03
2,399.0,2-2-241,2.288000e+03
3,49999.0,9-1-430,2.058700e+05
4,53999.0,9-2-4,2.058700e+05
...,...,...,...
478904,13999.0,25-1-1,9.569753e+12
478905,3499.0,25-1-1,9.795403e+12
478906,3125.0,5-1-3,9.990386e+12
478907,3125.0,5-1-3,9.990386e+12


In [213]:
# converting to string dtype y normalizando la cantidad numeros a 13 en la columna producto_id
precios_20200426["producto_id"]= precios_20200426["producto_id"].astype(str)
precios_20200426["producto_id"] = precios_20200426.producto_id.str.zfill(13)
# precios_20200426

# Cambio de las etiquetas en las columnas
precios_20200426 = precios_20200426.rename(columns={'sucursal_id': 'IdSucursal','producto_id' : 'IdProducto' })

#Agergamos la columna fecha
precios_20200426 = precios_20200426.assign (Fecha ="2020-04-26")

#Guardamos el archivos y cambiamos la codificación
# precios_20200426.to_csv ('.\Datasets_procesados\precios_semana_20200426_m.csv', index=False, sep=',', decimal='.', encoding= "utf_8")
producto.to_csv(r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\PIndividual\precios_semana_20200426_m.csv", index=False, sep=',', decimal='.', encoding= "utf_8")
precios_20200426

,precio,IdSucursal,IdProducto,Fecha
0,399.0,2-1-092,00000002288.0,2020-04-26
1,299.0,2-1-206,00000002288.0,2020-04-26
2,399.0,2-2-241,00000002288.0,2020-04-26
3,49999.0,9-1-430,00000205870.0,2020-04-26
4,53999.0,9-2-4,00000205870.0,2020-04-26
...,...,...,...,...
478904,13999.0,25-1-1,9569753142128.0,2020-04-26
478905,3499.0,25-1-1,9795403001143.0,2020-04-26
478906,3125.0,5-1-3,9990385651922.0,2020-04-26
478907,3125.0,5-1-3,9990385651939.0,2020-04-26


A continuación uniremos las tablas, en un tabla de precio

In [142]:
# precios = pd.concat([precios_20200413, precios_20200419,precios_20200426, precios_20200503], axis = 0)

# precios.to_csv ('.\Datasets_procesados\precios_m.csv', index=False, sep=',', decimal='.', encoding= "utf_8")

In [214]:
# Unimos las tablas de precios
precios = pd.concat([precios_20200413, precios_20200419,precios_20200426, precios_20200503], axis = 0)

#Realizamos la limpieza general de la tabla en la columna precio 
precios.precio.fillna(0, inplace= True)
precios["precio"].replace({'': 0}, inplace= True) 
# precios["precio"].iloc[246393]

# Se crea un archivo .csv con los datos normalizados
# precios.to_csv ('.\Datasets_procesados\precios_m.csv', index=False, sep=',', decimal='.', encoding= "utf_8")
producto.to_csv(r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\PIndividual\precios_m", index=False, sep=',', decimal='.', encoding= "utf_8")



In [2]:
import mysql.connector

cnn = mysql.connector.connect(host= "localhost", user = "root", passwd = "ShinigamI1", database = "proyecto_individual_w1")
cnn = mysql.connector.connect(host= "localhost", user = "root", passwd = "ShinigamI1", database = "proyecto_individual_w1")
cur= cnn.cursor()
cur.execute("SELECT AVG(precio) AS precio_promedio FROM precios WHERE IdSucursal = '9-1-688'")
datos = cur.fetchall()

datos

[(205.9753313794428,)]

In [ ]:
cur.close()
cnn.close()

In [219]:
from pandas.io import sql
import pandas as pd
import sqlalchemy as db
from datetime import datetime, date
import numpy as np
from scipy import stats
import pymysql

In [223]:
#Tabla que se incrementara
tabla_nueva_precios = pd.read_csv(".\Datasets\precios_semana_20200518.txt", sep= "|")
tabla_nueva_precios

,precio,producto_id,sucursal_id
0,29.90,0000000002288,2-1-009
1,32.90,0000000002288,2-1-037
2,36.90,0000000002288,2-1-090
3,39.90,0000000002288,2-3-247
4,499.99,0000000205870,9-1-430
...,...,...,...
415288,149.99,9569753142128,25-1-1
415289,34.99,9795403001143,25-1-1
415290,312.50,9990385651922,5-1-4
415291,312.50,9990385651939,5-1-4


In [224]:
# Convertimos a dataframe
tabla_nueva_precios = pd.DataFrame(tabla_nueva_precios)

# Normalizamos los datos de la tabla
# converting to string dtype
tabla_nueva_precios["producto_id"]= tabla_nueva_precios["producto_id"].astype(str)

# Separamos los datos de Id y creo la columna de IdProducto
tabla_nueva_precios[['0','1','IdProducto']] = tabla_nueva_precios.producto_id.str.rpartition('-', expand=True)
tabla_nueva_precios

# #Normalizamos la columna IdProducto a 13 dígitos
tabla_nueva_precios["IdProducto"] = tabla_nueva_precios["IdProducto"].str.zfill(13)

# Agregamos la columna de fecha
tabla_nueva_precios = tabla_nueva_precios.assign(Fecha ="2020-05-18")

# Cambiamos el nombre la columna sucursal_id
tabla_nueva_precios = tabla_nueva_precios.rename(columns={'sucursal_id': 'IdSucursal'})


# Se seleccionan las columnas
tabla_nueva_precios = tabla_nueva_precios[['precio', 'IdProducto', 'IdSucursal','Fecha']]
tabla_nueva_precios


,precio,IdProducto,IdSucursal,Fecha
0,29.90,0000000002288,2-1-009,2020-05-18
1,32.90,0000000002288,2-1-037,2020-05-18
2,36.90,0000000002288,2-1-090,2020-05-18
3,39.90,0000000002288,2-3-247,2020-05-18
4,499.99,0000000205870,9-1-430,2020-05-18
...,...,...,...,...
415288,149.99,9569753142128,25-1-1,2020-05-18
415289,34.99,9795403001143,25-1-1,2020-05-18
415290,312.50,9990385651922,5-1-4,2020-05-18
415291,312.50,9990385651939,5-1-4,2020-05-18


In [226]:
database_username='root' 
database_password='ShinigamI1'
database_ip='localhost'
database_name='proyecto_individual_w1'
database_conection=db.create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}')
coneccion=database_conection.connect()
metadata=db.MetaData()

tabla_nueva_precios.to_sql('precios_20200518_m', coneccion)

415293

In [ ]:
# Unimos las tablas de precios y realizamos la carga incremental
import pandas as pd
precios_reunidos= pd.concat([precios_20200413, precios_20200419,precios_20200426, precios_20200503, tabla_nueva_precios], axis = 0)
precios_reunidos.to_csv(r"C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\PIndividual\precios_reunidos", index=False, sep=',', decimal='.', encoding= "utf_8")

precios_reunidos.to_sql('precios_reunidos', coneccion)


